Imports:

In [1]:
import re
import sys
from nltk.tokenize import TweetTokenizer
from difflib import SequenceMatcher

# Tokeniser 1:

#### Sanna

In [2]:
def tokenise(line):
    # Initialise lists
    tokens = []
    unmatchable = []

    # Compile patterns for speedup
    token_pat = re.compile(r'\w+|#|\'|@')
    skippable_pat = re.compile(r'[.,]+')  # typically spaces

    # As long as there's any material left...
    while line:
        # Try finding a skippable token delimiter first.
        skippable_match = re.search(skippable_pat, line)
        if skippable_match and skippable_match.start() == 0:
            # If there is one at the beginning of the line, just skip it.
            line = line[skippable_match.end():]
        else:
            # Else try finding a real token.
            token_match = re.search(token_pat, line)
            #print(token_match)
            if token_match and token_match.start() == 0:
                #print(line[token_match.start():token_match.end()])
                if line[token_match.start():token_match.end()] == '#': #keep hash tags together and seperate
                    try:
                        token_match2 = re.search(token_pat, line[1:])
                        tokens.append(line[:token_match2.end()+1])
                        line = line[token_match2.end()+1:]
                    except:
                        line = line[token_match.end():]

                elif line[token_match.start():token_match.end()] == '@': #keep @ tags together and seperate
                    try:
                        token_match2 = re.search(token_pat, line[1:])
                        if ' ' in line[token_match2.start():token_match2.end()]:
                            line = line[token_match.end():]
                        
                        else: 
                            tokens.append(line[:token_match2.end()+1])
                            line = line[token_match2.end()+1:]
                    except:
                        line = line[token_match.end():]

                elif line[token_match.start():token_match.end()] == "'": #handle contractions as a single word
                    try:
                        token_match2 = re.search(token_pat, line[1:])
                        if ' ' in line[token_match2.start():token_match2.end()]:
                            line = line[token_match.end():]
                        
                        else: 
                            tokens.append(line[:token_match2.end()+1])
                            line = line[token_match2.end()+1:]
                    except:
                        line = line[token_match.end():]

                # If there is one at the beginning of the line, tokenise it.
                else:
                    tokens.append(line[:token_match.end()])
                    line = line[token_match.end():]
            else:
                # Else there is unmatchable material here.
                # It ends where a skippable or token match starts, or at the end of the line.
                unmatchable_end = len(line)
                if skippable_match:
                    unmatchable_end = skippable_match.start()
                if token_match:
                    unmatchable_end = min(unmatchable_end, token_match.start())
                # Add it to unmatchable and discard from line.
                unmatchable.append(line[:unmatchable_end])
                line = line[unmatchable_end:]

    final_tokens = []

    while len(tokens) > 0:
        temp1 = tokens.pop(0)
        try:
            temp2 = tokens.pop(0)
            if temp2[0] == "'":
                temp1 += temp2
                final_tokens.insert(0, temp1)
                #print('a', temp1)
            else:
                final_tokens.insert(0, temp1)
                tokens.insert(0, temp2)
                #print('b', temp1, temp2)
        except:
            final_tokens.insert(0, temp1)
            #print('d', temp1)
        
    final_tokens = final_tokens[::-1]

    #print(final_tokens)
    #print(unmatchable)
    return final_tokens

# Tokeniser 2:

#### Dee

In [3]:
def tokeniser2(line):
    #line = 'A cat sat on the mat. His name was Måns.'
    #line = 'friendship at its finest. ....#pixar #toystory #buzz #woody #friends #friendship #bff…'
    #line = "No I don't have any carrots!! #seattlepoloclub #horse #horses @ Seattle Polo &amp; Equestrian Club"

    # Initialise lists
    tokens = []
    unmatchable = []

    # Compile patterns for speedup
    token_pat = re.compile(r'#*[a-zA-Z0-9_]+')
    skippable_pat = re.compile(r'\s+')  # typically spaces

    # As long as there's any material left...
    while line:
        # Try finding a skippable token delimiter first.
        skippable_match = re.search(skippable_pat, line)
        if skippable_match and skippable_match.start() == 0:
            # If there is one at the beginning of the line, just skip it.
            line = line[skippable_match.end():]
        else:
            # Else try finding a real token.
            token_match = re.search(token_pat, line)
            if token_match and token_match.start() == 0:
                # If there is one at the beginning of the line, tokenise it.
                tokens.append(line[:token_match.end()])
                line = line[token_match.end():]
            else:
                # Else there is unmatchable material here.
                # It ends where a skippable or token match starts, or at the end of the line.
                unmatchable_end = len(line)
                if skippable_match:
                    unmatchable_end = skippable_match.start()
                if token_match:
                    unmatchable_end = min(unmatchable_end, token_match.start())
                # Add it to unmatchable and discard from line.
                unmatchable.append(line[:unmatchable_end])
                line = line[unmatchable_end:]

    return(tokens)

## Create a mini data set to test tokenisers against nltk.tokenize

In [4]:
f = open('../datasets/emoji/raw/train_text.txt', "r", encoding="utf-8")
final_emoji = []
n = 0 #change this to see a particular line

for line in f:
    final_emoji.append(line)
f.close()
token_test_set = final_emoji[0:20]
print(len(token_test_set))

token_test_set[n]

20


'Sunday afternoon walking through Venice in the sun with @user ️ ️ ️ @ Abbot Kinney, Venice \n'

# nltk.tokenize

In [5]:
tknzr = TweetTokenizer()
tokeniser_list = []

for i in token_test_set:
    tokeniser_list.append(tknzr.tokenize(i))
    
tokeniser_list[n]

['Sunday',
 'afternoon',
 'walking',
 'through',
 'Venice',
 'in',
 'the',
 'sun',
 'with',
 '@user',
 '️',
 '️',
 '️',
 '@',
 'Abbot',
 'Kinney',
 ',',
 'Venice']

# Tokeniser 1 test

In [6]:
tokeniser1_list = []
count = 0
ratio_tokeniser1 = []

for i in token_test_set:
    tokeniser1_list.append(tokenise(i))
    
    seq = SequenceMatcher(a=tokeniser_list[count], b=tokeniser1_list[count])
    ratio_tokeniser1.append(seq.ratio())
    count +=1

print(f'{sum(ratio_tokeniser1)/len(ratio_tokeniser1):.0%}')   
tokeniser1_list[n]

84%


['Sunday',
 'afternoon',
 'walking',
 'through',
 'Venice',
 'in',
 'the',
 'sun',
 'with',
 '@user',
 'Abbot',
 'Kinney',
 'Venice']

# Tokeniser 2 test

In [7]:
tokeniser2_list = []
count = 0
ratio_tokeniser2 = []

for i in token_test_set:
    tokeniser2_list.append(tokeniser2(i))
    
    seq = SequenceMatcher(a=tokeniser_list[count], b=tokeniser2_list[count])
    ratio_tokeniser2.append(seq.ratio())
    count +=1

print(f'{sum(ratio_tokeniser2)/len(ratio_tokeniser2):.0%}') 
tokeniser2_list[n]

82%


['Sunday',
 'afternoon',
 'walking',
 'through',
 'Venice',
 'in',
 'the',
 'sun',
 'with',
 'user',
 'Abbot',
 'Kinney',
 'Venice']